<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/analysingJsonoutputfromCVparser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [213]:
import json

# Assuming 'resume.json' is in a directory called 'resumes' in the same parent directory as your Python script
json_file_path1 = "/content/CV_parsed_by_CV_Parser_Premium.json"  # Relative path

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path1, 'r') as file:
    resume_data = json.load(file)

# Labeling the job entries
job_count = 1
for experience in resume_data['profile']['professional_experiences']:
    experience['job_label'] = f'Job_{job_count}'
    job_count += 1

# Save the modified data back to the JSON file
output_json_path = "/content/jobslabeled.json"
with open(output_json_path, 'w') as output_file:
    json.dump(resume_data, output_file, indent=2)

print(f"Job labeling completed. Labeled data saved to {output_json_path}")

# Now 'resume_data' contains the contents of the JSON file
# print(json.dumps(resume_data, indent=2))


Job labeling completed. Labeled data saved to /content/jobslabeled.json


In [214]:
import json
import pandas as pd

def json_to_dataframe(json_file_path):
    # Open the JSON file and load its contents into a Python dictionary
    with open(json_file_path, 'r') as file:
        resume_data = json.load(file)

    # Convert the labeled data into a DataFrame
    df = pd.json_normalize(resume_data['profile']['professional_experiences'])

    # Remove the 'description' column if it exists e.g. candidate blurb on the roles they've had
    if 'description' in df.columns:
        df = df.drop('description', axis=1)

    return df

if __name__ == "__main__":
    # Assuming 'jobslabeled.json' is in the same directory as your new script
    json_file_path2 = "jobslabeled.json"

    # Convert the labeled JSON data to a DataFrame
    df = json_to_dataframe(json_file_path2)

    # Save the DataFrame as a CSV file
    csv_output_path = "output_data.csv"
    df.to_csv(csv_output_path, index=False)

    # Display the DataFrame
    print(df)

   is_current  duration_in_months  \
0       False                   7   
1       False                  10   
2       False                   3   
3       False                   5   
4       False                  24   
5       False                   1   
6       False                   9   
7       False                   6   

                                       company  \
0                              EEI Corporation   
1                              EEI Corporation   
2              GS Construction Middle East LLC   
3                    Atlantic Gulf and Pacific   
4                GS ENGINEERING & CONSTRUCTION   
5  EASTWEST WORKS AND INDUSTRIAL SERVICES INC.   
6               Atlantic Gulf & Pacific (AG&P)   
7               Atlantic Gulf & Pacific (AG&P)   

                                  location           client        title  \
0  Brgy. Sta Rita, Batangas City, Batangas  McCONNEL DOWELL   Pipefitter   
1           Tanauan, Batangas, Philippines                    P

In [194]:
pip install dataprep

In [215]:
# from dataprep.clean import clean_country
import pandas as pd

# # Clean the 'location' column using clean_country
cleaned_df = clean_country(df, "location", input_format="official")
df = cleaned_df


# # Display the cleaned DataFrame
print(df)

# # output is location_cleaned column



/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
                                     

Country Cleaning Report:
	3 values cleaned (37.5%)
	5 values unable to be parsed (62.5%), set to NaN
Result contains 3 (37.5%) values in the correct format and 5 null values (62.5%)
   is_current  duration_in_months  \
0       False                   7   
1       False                  10   
2       False                   3   
3       False                   5   
4       False                  24   
5       False                   1   
6       False                   9   
7       False                   6   

                                       company  \
0                              EEI Corporation   
1                              EEI Corporation   
2              GS Construction Middle East LLC   
3                    Atlantic Gulf and Pacific   
4                GS ENGINEERING & CONSTRUCTION   
5  EASTWEST WORKS AND INDUSTRIAL SERVICES INC.   
6               Atlantic Gulf & Pacific (AG&P)   
7               Atlantic Gulf & Pacific (AG&P)   

                                 

In [217]:
# this code works, butdoesnt handle countries in the location column.
# Uae is an example which is hardcoded in.
# to get around this I;ve also used clean_country to get Kuwait, but I didn't know what the problem was when I inititated this.
# anyway to keep using this code it needs to be solved.


# The other option is to use the script below this one that uses GEOPY - try looking into this!!!!!



# Assuming your DataFrame is named df
df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)

# Create a mapping between 'City' and 'Country' columns in df1
mapping_city_country = dict(zip(df1['City'], df1['Country']))

# Create a pattern for regex using city names
pattern = '|'.join([rf'\b{city}\b' for city in df1['City']])

# Use str.contains with regex pattern to check for city names in 'location' column
df['contains_city'] = df['location'].str.contains(pattern, case=False, na=False)

# If a city is found, map the corresponding country using the mapping
df['country_matched_city'] = df['location'].apply(lambda x: next((country for city, country in mapping_city_country.items() if pd.notna(x) and city.lower() in x.lower()), pd.NA))

# Check if 'uae' is a full word in the 'location' column and assign the country directly
uae_present = df['location'].str.contains(r'\buae\b', case=False, na=False)
df.loc[uae_present, 'country_matched_city'] = 'UAE'

# Create a new column 'combined_location'
df['combined_location'] = df.apply(lambda row: row['location_clean'] if pd.notna(row['location_clean']) else row['country_matched_city'], axis=1)

# Capitalize the first letter and make all letters capitals if the country is 4 letters or less
df['combined_location'] = df['combined_location'].apply(lambda x: x.capitalize() if pd.notna(x) and len(x) > 4 else x.upper() if pd.notna(x) else pd.NA)

# Display the updated DataFrame
print(df)


   is_current  duration_in_months  \
0       False                   7   
1       False                  10   
2       False                   3   
3       False                   5   
4       False                  24   
5       False                   1   
6       False                   9   
7       False                   6   

                                       company  \
0                              EEI Corporation   
1                              EEI Corporation   
2              GS Construction Middle East LLC   
3                    Atlantic Gulf and Pacific   
4                GS ENGINEERING & CONSTRUCTION   
5  EASTWEST WORKS AND INDUSTRIAL SERVICES INC.   
6               Atlantic Gulf & Pacific (AG&P)   
7               Atlantic Gulf & Pacific (AG&P)   

                                  location           client        title  \
0  Brgy. Sta Rita, Batangas City, Batangas  McCONNEL DOWELL   Pipefitter   
1           Tanauan, Batangas, Philippines                    P

<ipython-input-217-4455c4291350>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)
<ipython-input-217-4455c4291350>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['contains_city'] = df['location'].str.contains(pattern, case=False, na=False)


In [219]:
# code using geopy
# see if we can get rid out country_clean as a result????????


from geopy.geocoders import Nominatim

# Assuming your DataFrame is named df
df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)

# Create a mapping between 'City' and 'Country' columns in df1
mapping_city_country = dict(zip(df1['City'], df1['Country']))

# Create a pattern for regex using city names
pattern = '|'.join([rf'\b{city}\b' for city in df1['City']])

# Use str.contains with regex pattern to check for city names in 'location' column
df['contains_city'] = df['location'].str.contains(pattern, case=False, na=False, regex=False)

# Initialize geolocator
geolocator = Nominatim(user_agent="geo_locator")

# Function to get country based on location
def get_country(location):
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.address.split(",")[-1].strip()
        else:
            return pd.NA
    except:
        return pd.NA

# Apply the function to get countries based on location
df['country_matched_city'] = df['location'].apply(get_country)

# Display the updated DataFrame
print(df)


<ipython-input-219-55c69ed96ab3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)


   is_current  duration_in_months  \
0       False                   7   
1       False                  10   
2       False                   3   
3       False                   5   
4       False                  24   
5       False                   1   
6       False                   9   
7       False                   6   

                                       company  \
0                              EEI Corporation   
1                              EEI Corporation   
2              GS Construction Middle East LLC   
3                    Atlantic Gulf and Pacific   
4                GS ENGINEERING & CONSTRUCTION   
5  EASTWEST WORKS AND INDUSTRIAL SERVICES INC.   
6               Atlantic Gulf & Pacific (AG&P)   
7               Atlantic Gulf & Pacific (AG&P)   

                                  location           client        title  \
0  Brgy. Sta Rita, Batangas City, Batangas  McCONNEL DOWELL   Pipefitter   
1           Tanauan, Batangas, Philippines                    P